# Time-varying AR(1) exercise

> Joris de Wind (1:1 Analytics, DBNL)

### Model description

In this exercise, you will learn about autoregressive models with time-varying parameters. In particular, we will use a Bayesian approach to estimate the time-varying AR(1) model

> $y_t = \mu_t + \varrho_t y_{t-1} + \varepsilon_t$
with $\varepsilon_t \sim N(0, R)$

where the coefficients are assumed to follow a random walk, that is

> $\theta_t = [\mu_t, \varrho_t]' = \theta_{t-1} + \upsilon_t$
with $\upsilon_t \sim N(0, Q)$

The covariance matrix $Q$ is very important, as it controls the amount of time variation in the parameters. The larger $Q$ the larger the variance of the changes in the time-varying parameters.

### Example and some explanation

This model (and in particular its multivariate generalizations) can for example be used to model inflation. The persistence and target level of inflation can be influenced by changes in monetary policy, which will be reflected in changes in $\varrho_t$ and $\mu_t$, respectively. As a matter of fact, if the central bank becomes more effective the persistence of the inflation process will drop, i.e. the economy will recover faster after a shock to the economy.

Since the data can be explained both by temporary shocks $\varepsilon_t$ as well as permanent structural changes to the system $\upsilon_t$, we need to let the data speak which of the two explanations is more realistic. The most likely estimate is typically a combination of the two, but the weights depend on the circumstances and it's a _dynamic_ process. For example, a lower inflation rate might at first be explained by temporary shocks but if inflation continues to be low for a longer period it will be a good idea to revise this assessment and consider the possibility that the central bank actually lowered its target inflation rate.

### Estimation procedure

To estimate this model we will use the __Gibbs sampler__ as well as the __Kalman filter__. But don't worry: the goal of this exercise is to teach you about the structure of the __Bayesian estimation__ procedure rather than the tiny details.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import invwishart
from scipy.linalg import solve, cholesky
from numba import jit

## Part 1: generating coefficients and fake data

For illustration purposes, we will work with fake data in this exercise. This enables us to compare the estimated trajectory for the time-varying parameters with the true trajectory with which the data was generated.

You can just run this part of the code, i.e. there are no exercises in part 1. Of course, you are more than welcome to come back to this part after you have successfully finished all the exercises in the remainder of this notebook.

In particular, it would be interesting to _experiment_ with a different trajectory for the parameters, and study the consequences on the accuracy of the estimated trajectory. For example, you could study the consequences when the parameter change is not gradual (as in the code below) but instead very abrupt. Or, you could estimate the model on real data!

**Okay, one non-programming exercise after all**: can you explain the connection between the two plots generated below? In particular, do you understand the effects of the parameter changes on the simulated data?

In [ ]:
# generating evolution of coefficients
number_of_periods = 240
rho = np.zeros(number_of_periods)
rho[0:80] = 0.9
rho[80:130] = 0.9 + (0.5 - 0.9) * 1 / (1 + np.exp(-0.25 * np.arange(-25, 25)))
rho[130:240] = 0.5
const = 1 - rho  # to keep the unconditional mean constant
sig = 0.007

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(rho, color='r', linewidth=3, label=r"$\rho$")
plt.plot(const, color='b', linewidth=3, label="constant")
plt.title("Evolution of coefficients", fontsize=16, fontweight='bold')
plt.xlabel("t", style='italic')
plt.legend(fontsize=14)
txt = \
"Back to the central banking example,\n" + \
"around period 100 a very smart data\n" + \
"scientist is hired by the central bank.\n\n" + \
"This data scientist can build models to\n" + \
"design more effective monetary policy,\n" + \
"making the economy more resilient to\n" + \
"shocks and therefore decreasing the\n" + \
"persistence of inflation.\n\n" + \
"Some smart market watchers were able\n" + \
"to anticipate the drop in persistence,\n" + \
"which led to a drop in persistence even\n" + \
"before the actual increase in policy\n" + \
"effectiveness! Of course, there are\n" + \
"always people with a delayed\nresponse."
plt.text(-2, 0.25, txt, fontsize=12.8, style='italic')
plt.axvline(x=100, linestyle='--')
plt.show()

In [ ]:
# generating fake data
np.random.seed(seed=179)
data = np.zeros(number_of_periods)
data[0] = const[0] / (1 - rho[0])  # start at the unconditional mean
for i in range(1, number_of_periods):
    data[i] = const[i] + rho[i] * data[i-1] + sig * np.random.randn()

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(data, color='k', linewidth=3)
plt.title("Fake data", fontsize=16, fontweight='bold')
plt.xlabel("t", style='italic')
txt = "Can you guess in which part of the sample\nperiod the data was more persistent?"
plt.text(80, 1.03, txt, fontsize='14', style='italic')
plt.show()

## Part 2: preparing the data and choosing priors

Before running the Bayesian estimation procedure, we need to decide which priors to use. One of the standard approaches is to use a training sample, and use time-invariant OLS on the training sample to calibrate the priors. Of course, it's not allowed to recycle the training sample.

In [ ]:
# preparing the data
p = 1
T = len(data) - p
k = 1 + p  # number of time-varying parameters (constant + autoregressive)
y = data[p:]
X = np.ones((T, 2))
X[:, 1] = data[:-p]

## Exercise 1 - splitting the data

Split the data, i.e. take the first 40 observations as training sample and the remainder as the sample that's going to be used in the estimation

In [ ]:
# splitting the data (we use a training sample to calibrate the prior)
yp = XXX  # for training sample
Xp = XXX  # for training sample
y = XXX   # for regular sample
X = XXX   # for regular sample
T = len(y)

In [ ]:
# OLS on training sample
ols = sm.OLS(yp, Xp).fit()  # time-invariant OLS on training sample
                            # to calibrate the prior

## Exercise 2 - setting the priors

We need to come up with priors for

- $\theta_0$
- $Q$
- $R$

The prior for $R$ is already taken care of, free of charge! So, let's focus on the priors for $\theta_0$ and $Q$. Note that the prior for $\theta_0$ together with the prior for $Q$ automatically imply a prior for the entire trajectory of $\theta_t$ (through the random walk equation). **Make sure you understand this, and otherwise ask your neighbor!**

We will use the OLS estimates on the training sample to calibrate the priors for both $\theta_0$ and $Q$. Based on the training sample, we have a rough idea for the appropriate level of $\theta_0$ and we can use the OLS estimation uncertainty (i.e. the covariance matrix of the parameter estimation errors) as a basis for the prior on the amount of time variation, which is parameterized by $Q$.

Anyway, regarding $\theta_0$ we want to use a normally distributed prior with

- mean equal to the time-invariant OLS estimate on the training sample
- variance proportional to the covariance matrix of the parameter estimation errors

and regarding $Q$ we want to use an inverse-Wishart distribution with$^1$

- scale matrix proportional to the covariance matrix of the parameter estimation errors
- degrees-of-freedom set to k+1, i.e. the number of time-varying parameters plus one

$^1$Don't worry if you have never heard about the inverse-Wishart distribution, it's a multivariate generalization of the inverse-Gamma distribution (with some additional restrictions), and it's used quite often in Bayesian statistics. In any case, the inverse-Wishart distribution is parameterized by a scale matrix and degrees-of-freedom parameter. The mean and mode of the inverse-Wishart distribution are proportional to the scale matrix and the variances of its elements are decreasing in the degrees-of-freedom parameter.

### Exercises

1. Set `THbar` and `PeK[:, :, 0]` to the above discussed prior mean and variance for $\theta_0$, respectively. Note that PeK is a 3D-array and the rest of this array is filled with zeros. Don't worry about that, that's just for memory allocation.

2. Set `Qbar` and `T0Q` to the above discussed scale matrix and degrees-of-freedom for $Q$, respectively.

In [ ]:
# prior on the time-varying parameters
THbar = XXX
PeK = np.zeros((k, k, T+1))
PeK[:, :, 0] = 4 * XXX

# prior on the shocks to the parameters
Qbar = 0.1 * XXX
T0Q = XXX

# prior on the covariance matrix
Rbar = ols.mse_resid
T0R = 2

## Part 3: Bayesian estimation using Gibbs sampling

Here is where the magic happens. We are interested in estimating the joint posterior density$^1$

> $P(\theta^T, Q, R|y^T)$

but unfortunately we don't have an analytical expression for this density. This is where the Gibbs sampler comes to the rescue.

With the Gibbs sampler we basically iterate between the various conditional posterior densities, and under some regularity conditions (which are satisfied in this case) this process will converge to the joint posterior density.

In the current case, the Gibbs sampler is very useful because we can derive analytical expressions for the various conditional posterior densities:

- $P(\theta^T|Q, R, y^T)$
- $P(Q|\theta^T, R, y^T)$
- $P(R|\theta^T, Q, y^T)$

Regarding $P(\theta^T|Q, R, y^T)$, for given covariance matrices $Q$ and $R$ we simply have a fully parameterized state space model. And we therefore need the Kalman filter ... Well, the Kalman filter yields point estimates, and we actually need to run the Kalman simulation smoother to draw random samples. You don't need to program this step yourself, i.e. the simulation smoother function will be provided below (we did the hard work for you!).

Regarding $P(Q|\theta^T, R, y^T)$, for given $\theta^T$ we can calculate the realized shocks to the time-varying parameters (i.e. take the first difference of $\theta_t$), from which we can calculate the sum-of-squared residuals. The sum-of-squared residuals form the basis to simulate a random covariance matrix using the inverse-Wishart distribution.

Regarding $P(R|\theta^T, Q, y^T)$, for given $\theta^T$ we can calculate the realized shocks in the AR(1) equation (i.e. simply use $\varepsilon_t = y_t - \mu_t - \varrho_t y_{t-1}$), from which we can calculate the sum-of-squared residuals. The sum-of-squared residuals form the basis to simulate a random covariance matrix using the inverse-Wishart distribution.

$^1$With superscript $x^T$ we mean to include the entire trajectory of $x_t$ from $x_0$ up to $x_T$ inclusive.

In [ ]:
# create simulation smoother function (needed in Gibbs sampler below)
@jit
def sampleTH(Q, R, Pe, TH):
    # Kalman filter
    Po = np.zeros((k, k, T))  # memory allocation
    for t in range(T):
        Xtemp = X[t, :].reshape(1, -1)
        Po[:, :, t] = Pe[:, :, t] + Q
        K = solve(Xtemp @ Po[:, :, t] @ Xtemp.T + R,   # note that solve is numerically more
                  Xtemp @ Po[:, :, t]).reshape(-1, 1)  # stable than directly using inv
        Pe[:, :, t+1] = Po[:, :, t] - K @ Xtemp @ Po[:, :, t]
        TH[:, t+1] = TH[:, t] + K @ (y[t] - Xtemp @ TH[:, t])
    # Kalman simulation smoother
    TH[:, T] = TH[:, T] + cholesky(Pe[:, :, T]).T @ \
               np.random.randn(k)
    for t in reversed(range(T)):
        temp1 = Pe[:, :, t]
        temp2 = solve(Po[:, :, t], temp1).T
        TH[:, t] = TH[:, t] + temp2 @ (TH[:, t+1] - TH[:, t]) + \
                   cholesky(temp1 - temp2 @ temp1).T @ \
                   np.random.randn(k)
    return TH

## Exercise 3 - complete the Gibbs sampler

You will basically need to complete the for loop below.

1. To draw a new trajectory for $\theta^T$, you will need to call the `sampleTH` function with the most recent draws for $Q$ and $R$. Note that you will also need to provide the arguments `PeK` and `TH[:, :, i]`.
2. To draw a new covariance matrix $Q$, you will need to calculate the realized shocks to the time-varying parameters (`v` in the code below) based on the most recent trajectory for $\theta^T$. After calculating `v`, you can calculate the sum-of-squared residuals, which will in turn be used in the `invwishart.rvs` sampling function.
3. To draw a new covariance matrix $R$, you will need to calculate the realized shocks in the AR(1) equation (`u` in the code below). After this, you can calculate the sum-of-squared residuals, which will in turn be used in the `invwishart.rvs` sampling function.

In [ ]:
# Bayesian estimation
g = 500  # length of Markov chain
b = 0.2  # burn-in fraction

# allocating memory and initialization
TH = np.zeros((k, T+1, g))
Q1 = np.zeros((k, k, g+1))
R1 = np.zeros((1, 1, g+1))
Q1[:, :, 0] = Qbar
R1[:, :, 0] = Rbar

# Gibbs sampler
for i in range(g):

    # sample TH (given Q1 and R1)
    TH[:, 0, i] = THbar
    TH[:, :, i] = XXX

    # sample Q1 (given TH and R1)
    v = XXX
    SSRv = XXX
    Qtemp = SSRv + T0Q * Qbar
    Qtemp = (Qtemp + Qtemp.T) / 2  # to improve numerical stability
    Q1[:, :, i+1] = invwishart.rvs(T + T0Q, Qtemp)

    # sample R1 (given TH and Q1)
    u = XXX
    SSRu = XXX
    Rtemp = SSRu + T0R * Rbar
    Rtemp = (Rtemp + Rtemp.T) / 2  # to improve numerical stability
    R1[:, :, i+1] = invwishart.rvs(T + T0R, Rtemp)

    # check progress
    if (i + 1) % (g / 10) == 0:
        print(f"Progress report: {100 * (i + 1) / g}% done")

##  Part 4: plotting the results

After finishing all the exercises above you can just run the code below and see how well the estimated trajectory tracks the true one. Of course, there is still quite some sampling uncertainty left but you did a good job, didn't you?

Don't celebrate too early, there are still several extra exercises below!

In [ ]:
TH_estimated = np.median(TH[:, :, int(b*g):], axis=2)  # discard burn-in samples
TH_estimated = np.concatenate((np.tile(ols.params, (int(ols.nobs), 1)).T,
                              TH_estimated), axis=1)

plt.figure(figsize=(12, 8))
plt.plot(rho, color='r', linewidth=3, label="true value")
plt.plot(TH_estimated[1, :], color='b', linewidth=3, label="estimated")
plt.title(r"Evolution of $\rho$", fontsize=16, fontweight='bold')
plt.xlabel("t", style='italic')
plt.legend(fontsize=14)
plt.show()

## Extra exercises - you need to finish at least one before you get the password for free drinks!

Feel free to pick one or two exercises that you find most interesting.

1. Adjust the code so as to estimate a time-varying AR(2) model. Tip: you only need to change the code of a single cell!
2. Play around with the constant-of-proportionality in the prior scale matrix for $Q$, and study the consequences on the estimated amount of time variation
3. Go back to part 1 and adjust the underlying trajectory of the parameters, and study the consequences on the accuracy of the estimated trajectory
4. Adjust the code to be able to run multiple Markov chains (i.e. multiple instances of the Gibbs sampler), and compare the chains to check the convergence of the Bayesian estimation procedure